In [1]:
from processors import *
import warnings, inspect
from coffea import processor
from coffea.nanoevents.methods import vector, candidate
from coffea.nanoevents import BaseSchema
#warnings.filterwarnings("ignore")

In [2]:
# prefix = '/eos/uscms/store/user/christiw/bparking/V1p19/MC_Fall18/v1/v9/normalized/'

# filenames = [
#     'BToKPhi_MuonGenFilter_PhiToPi0Pi0_mPhi0p3_ctau1000_1pb_weighted.root',
#     'BToKPhi_MuonGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_1pb_weighted.root',
#     'BToKPhi_MuonGenFilter_PhiToPiPlusPiMinus_mPhi0p3_ctau1000_1pb_weighted.root',
#     'BToKPhi_MuonGenFilter_PhiToPiPlusPiMinus_mPhi0p3_ctau300_1pb_weighted.root',
# ]

# fileset = {
#     f"{file.split('_')[2]}_{file.split('_')[3]}_{file.split('_')[4]}": 
#     [prefix+file] 
#     for file in filenames
# }

# fileset['background'] = ['/eos/uscms/store/user/christiw/bparking/V1p19/Data2018_UL/v9/normalized/ParkingBPH4_2018A_goodLumi.root']

#=======================================================================

prefix = 'root://cmseos.fnal.gov//store/user/ahayrape/BigNtupler/'

fileset = {
            'PhiToPi0Pi0_mPhi0p3_ctau300':
                [prefix + 'PhiToPi0Pi0_mPhi0p3_ctau300.root'],

            'PhiToPi0Pi0_mPhi1p0_ctau1000':
                [prefix + 'PhiToPi0Pi0_mPhi1p0_ctau1000.root'],

            'PhiToPiPlusPiMinus_mPhi0p3_ctau300':
                [prefix + 'PhiToPiPlusPiMinus_mPhi0p3_ctau300.root'],

            'PhiToPiPlusPiMinus_mPhi1p0_ctau1000':
                [prefix + 'PhiToPiPlusPiMinus_mPhi1p0_ctau1000.root'],

            'PhiToPiPlusPiMinus_mPhi1p0_ctau300':
                [prefix + 'PhiToPiPlusPiMinus_mPhi1p0_ctau300.root'],
         }

# fileset['background'] = [prefix + 'Background.root']
fileset['background'] = ['/eos/uscms/store/user/christiw/bparking/V1p19/Data2018_UL/v9/normalized/ParkingBPH4_2018A_goodLumi.root']


In [3]:
processors = {
    # 'ProcessorV1': ProcessorV1(),
    #'ProcessorV2': ProcessorV2(),
    'ProcessorV3': ProcessorV3('DT'),
}


test_mode = False

for proc in processors:
    print(proc)
    if test_mode:
        out = processor.run_uproot_job(
            fileset,
            treename="MuonSystem",
            processor_instance=processors[proc],
            executor=processor.futures_executor,
            # executor_args={"schema": BaseSchema, "workers": 10},
            executor_args={"schema": BaseSchema, "workers": 1},
            maxchunks = 1,
            chunksize=1,
        )
    else:
        out = processor.run_uproot_job(
            fileset,
            treename="MuonSystem",
            processor_instance=processors[proc],
            executor=processor.futures_executor,
            executor_args={"schema": BaseSchema, "workers": 10},
            # executor_args={"schema": BaseSchema, "workers": 1},
            # maxchunks = 200,
            # chunksize=10000,
        )
        
        filename = f'hists_cutflows_{proc}_04_17_23.pickle'
        outfile = open(filename, 'wb')
        pickle.dump(out, outfile)
        outfile.close()

ProcessorV3


Output()

Output()

/uscms/home/aaportel/nobackup/mamba/envs/bpllp/lib/python3.11/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch nDtRechits in <TTree 'MuonSystem' (209 branches) at 0x7f27d7a60cd0>, taking first instance
  warnings.warn(
/uscms/home/aaportel/nobackup/mamba/envs/bpllp/lib/python3.11/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch nDtRechits in <TTree 'MuonSystem' (209 branches) at 0x7f27d7a60cd0>, taking first instance
  warnings.warn(
/uscms/home/aaportel/nobackup/mamba/envs/bpllp/lib/python3.11/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch nDtRechits in <TTree 'MuonSystem' (209 branches) at 0x7f27d7a60cd0>, taking first instance
  warnings.warn(
/uscms/home/aaportel/nobackup/mamba/envs/bpllp/lib/python3.11/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch nDtRechits in <TTree 'MuonSystem' (209 branches) at 0x7f27d7a60cd0>,

In [ ]:
[(thing,out['background'][thing]) for thing in out['background'] if 'numEvents' in thing and '_SF' not in thing]